In [1]:
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import TypedDict, Annotated
import faiss
from langchain_community.vectorstores import FAISS

## **Prepare Vector Store**

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/attentation-all-you-need.pdf")
docs = loader.load()
len(docs)

15

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 80
)

documents = splitter.split_documents(documents=docs)
len(documents)

96